In [1]:
import requests
# import json
 
from PIL import Image
from pyquery import PyQuery as pq
from requests import RequestException
import csv
from wordcloud import WordCloud, STOPWORDS
import numpy as np
# import matplotlib.pyplot as plt
import jieba
import re

In [2]:
def get_one(page):
    """
    爬取一页
    :param page: start数
    :return: html源码
    """
    try:
        headers = {
            # 'Cookie': 'bid=kCNfNn6nvxA; __utmz=30149280.1564815097.1.1.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __yadk_uid=CvRkxvch6yW4IwpqONlyGzPCB1GznXLV; douban-fav-remind=1; ap_v=0,6.0; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1568964489%2C%22https%3A%2F%2Fwww.baidu.com%2Flink%3Furl%3D2p9zSFnHJuYwVjtaUY6Smc_R8lvOe0b5EatQr75zQywo5qIDZ1s5EUIST8Jw0K3qpAoQXJLWDJHTyx6jNbRq-a9MIbUXwkq4V8vrhxYGdqO%26wd%3D%26eqid%3Dd4a8230500089ac3000000055d452e46%22%5D; _pk_id.100001.8cb4=ef75a07b06bad72b.1564815095.2.1568964489.1564815095.; _pk_ses.100001.8cb4=*; __utma=30149280.546149049.1564815097.1564815097.1568964491.2; __utmc=30149280; __utmt=1; __utmb=30149280.3.8.1568964491',
            'Host': 'www.douban.com',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',  # 浏览器标识
        }
        url = 'https://www.douban.com/group/topic/297480010/?start={}'.format(page)  # url
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
    except RequestException:
        return None

In [3]:
def parse_one(html):
    """
    解析一页源码
    :param html:html源码
    :return:返回生成器，用于迭代
    """
    doc = pq(html)  # 初始化
    items = doc('#comments li .reply-doc').items()  # css选择器选择所有该标签
    # print(items.size())  # 评论的个数
    for item in items:
        yield {
            'name': item.find('h4 a').text(),  # 用户名
            'address': item.find('h4 a').attr('href'),  # 用户主页
            # 'signature': item.find('h4').text(),  # 个性签名
            'time': item.find('h4 span').text(),  # 回复时间
            'reply': item.find('p').text(),  # 回复内容
            'quote-reply': item.find('.reply-quote .short').text(),  # 引用的回复内容
            'quote-user': item.find('.reply-quote .pubdate').text(),  # 引用的用户名
            'quote-user-addr': str(item.find('.reply-quote .pubdate a').attr('href')).strip() if item.find('.reply-quote .pubdate a').attr('href') else '',  # 引用的用户的主页
            # 'vote': item.find('.comment-vote').text()  # 赞
        }

In [4]:
def write_csv(contents):
    """
    写入数据到csv文件
    :param contents: 解析的内容，可迭代
    :return:
    """
    with open('douban.csv', 'a', encoding='utf-8') as f:  # 追加方式写数据
        fieldnames = ['name', 'address', 'time', 'reply', 'quote-reply', 'quote-user', 'quote-user-addr']  # 第一行
        write = csv.DictWriter(f, fieldnames=fieldnames, lineterminator='\n')  # 消除多余的换行，不加lineterminator会出现每写一行空一行的情况。
        write.writeheader()
        for content in contents:
            print(type(content))
            write.writerow(content)

In [5]:
def read_csv():
    """
    将回复的内容保存在列表中并返回
    :return: 
    """
    reply = []
    n = 0
    with open('douban.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if row[4] != 'quote-reply':
                reply.append(row[3])
    return reply

In [6]:
for i in range(15):
    write_csv(parse_one(get_one(i * 100)))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di

In [13]:
import csv
from collections import Counter
import jieba

# 读取CSV文件并提取文本数据
filename = 'douban.csv'  # 替换为您的CSV文件名
column_index = 3  # 文本数据在CSV文件的第四列，索引从0开始

with open(filename, 'r', encoding='gbk') as file:
    reader = csv.reader(file)
    text_data = [row[column_index] for row in reader]

# 分词并过滤停用词和标点符号
seg_list = []
for text in text_data:
    seg_list.extend(jieba.cut(text))

# 统计词频
word_count = Counter(seg_list)

# 排序词频
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# 保存结果到文件
output_filename = 'word_frequency.csv'  # 替换为您要保存的文件名

with open(output_filename, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Word', 'Frequency'])  # 写入表头
    writer.writerows(sorted_word_count)  # 写入词频数据

print('词频统计结果已保存到文件:', output_filename)

词频统计结果已保存到文件: word_frequency.csv


In [14]:
import csv
import re
from collections import Counter
import jieba

# 读取CSV文件并提取文本数据
filename = 'douban.csv'  # 替换为您的CSV文件名
column_index = 3  # 文本数据在CSV文件的第四列，索引从0开始

with open(filename, 'r', encoding='gbk') as file:
    reader = csv.reader(file)
    text_data = [row[column_index] for row in reader]

# 定义无意义词和中文标点符号的正则表达式
pattern = re.compile(r'[\u4e00-\u9fa5a-zA-Z0-9]')  # 匹配中文、英文字母和数字

# 分词并过滤停用词和标点符号
seg_list = []
for text in text_data:
    words = jieba.cut(text)
    seg_list.extend([word for word in words if pattern.match(word)])

# 统计词频
word_count = Counter(seg_list)

# 排序词频
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# 保存结果到文件
output_filename = 'word_frequency.csv'  # 替换为您要保存的文件名

with open(output_filename, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Word', 'Frequency'])  # 写入表头
    writer.writerows(sorted_word_count)  # 写入词频数据

print('词频统计结果已保存到文件:', output_filename)

词频统计结果已保存到文件: word_frequency.csv


In [15]:
import csv
import re
from collections import Counter
import jieba

# 读取CSV文件并提取文本数据
filename = 'douban.csv'  # 替换为您的CSV文件名
column_index = 3  # 文本数据在CSV文件的第四列，索引从0开始

with open(filename, 'r', encoding='gbk') as file:
    reader = csv.reader(file)
    text_data = [row[column_index] for row in reader]

# 定义无意义词和中文标点符号的正则表达式
pattern = re.compile(r'[\u4e00-\u9fa5a-zA-Z0-9]')  # 匹配中文、英文字母和数字

# 加载停用词列表
jieba.analyse.set_stop_words('cn_stopwords.txt')  # 替换为您的停用词文件路径

# 分词并过滤停用词和标点符号
seg_list = []
for text in text_data:
    words = jieba.cut(text)
    seg_list.extend([word for word in words if pattern.match(word) and word not in jieba.analyse.stop_words])

# 统计词频
word_count = Counter(seg_list)

# 排序词频
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# 保存结果到文件
output_filename = 'word_frequency.csv'  # 替换为您要保存的文件名

with open(output_filename, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Word', 'Frequency'])  # 写入表头
    writer.writerows(sorted_word_count)  # 写入词频数据

print('词频统计结果已保存到文件:', output_filename)

AttributeError: module 'jieba' has no attribute 'analyse'

In [17]:
import csv
import re
from collections import Counter
import jieba

# 读取CSV文件并提取文本数据
filename = 'douban.csv'  # 替换为您的CSV文件名
column_index = 3  # 文本数据在CSV文件的第四列，索引从0开始

with open(filename, 'r', encoding='gbk') as file:
    reader = csv.reader(file)
    text_data = [row[column_index] for row in reader]

# 定义无意义词和中文标点符号的正则表达式
pattern = re.compile(r'[\u4e00-\u9fa5a-zA-Z0-9]')  # 匹配中文、英文字母和数字

# 加载停用词列表
stopwords = set()
with open('cn_stopwords.txt', 'r', encoding='utf-8') as file:  # 替换为您的停用词文件路径和编码方式
    for line in file:
        stopwords.add(line.strip())

# 分词并过滤停用词和标点符号
seg_list = []
for text in text_data:
    words = jieba.lcut(text)
    seg_list.extend([word for word in words if pattern.match(word) and word not in stopwords])

# 统计词频
word_count = Counter(seg_list)

# 排序词频
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# 保存结果到文件
output_filename = 'word_frequency.csv'  # 替换为您要保存的文件名

with open(output_filename, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Word', 'Frequency'])  # 写入表头
    writer.writerows(sorted_word_count)  # 写入词频数据

print('词频统计结果已保存到文件:', output_filename)

词频统计结果已保存到文件: word_frequency.csv
